In [ ]:
#Submitted version

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import time
import snap
from matplotlib.font_manager import FontProperties
from sklearn.neighbors.kde import KernelDensity


# Fonctions

In [ ]:
def Entropy(X):
    """ estimate entropy of the density
    of the sample of the X[i,:]
    Input:
        X: matrix. Each row represents an iid realization of the density"""
    
    kde = KernelDensity(kernel='gaussian').fit(X)
    return np.mean(kde.score_samples(X))


def prox_edge(theta,gamma,i,j):
    x = theta.copy()
    if np.abs(theta[i] - theta[j]) < 2 * gamma :
        x[i] = np.mean([theta[i],theta[j]])
        x[j] = np.mean([theta[i],theta[j]])
    else:
        x[i] -= gamma * np.sign(x[i] - x[j])
        x[j] -= gamma * np.sign(x[j] - x[i])
    return x
    


def quadratic_loss(y, x):
    """
    """
    return 1. / 2. * np.sum((y-x)**2)


def gradient(G):
    """
    compute gradient of a graph
    """
    N = G.GetNodes()
    M = G.GetEdges()

    grad = np.zeros((M, N))

    for (i, e) in enumerate(G.Edges()):
        s = e.GetSrcNId()
        d = e.GetDstNId()
        grad[i, s], grad[i, d] = -1. , 1.
    return grad



def Seuillage_Dur(v,seuil = 1.0):
    y = np.copy(v)
    for i in range(len(v)):
        if np.abs(v[i]) > seuil:
            y[i] = np.sign(v[i])*seuil
    return y    


def U(theta,G,temp,lmbda):
    return temp * 0.5 * np.linalg.norm(y - theta) **2 + temp * lmbda * TV(theta,G)
    
    

def proxtv(y, grad, lmbda, it = 100, pas = 0.001):
    M, N = grad.shape
    u = np.zeros(M)  
    for i in range(it):
        u = u - 0.001 * (np.dot(grad,np.dot(grad.T,u)) - np.dot(grad,y))
        u = Seuillage_Dur(u,lmbda)
    return y - np.dot(grad.T,u)
        
        
        
        
    

def PLA(y, n_iter, gamma, lmbda, grad, temp, itprox = 100, pasprox = 0.001, gaussian_noise=True):
    """ ProxLA algorithm
    Inputs:
        n_iter: number of iterations
        y: signal on the original graph
        gamma: constant step size
        lmbda: weight of regularization
        grad : graph gradient matrix
        gaussian_noise: if False passty algorithm is applied, else Passty-Langevin
     Output:
         samples
    """
    
    M, N = G.GetEdges(), G.GetMxNId() #number of edges and vertices

    samples_theta = np.zeros((n_iter, N)) #iterates stacked by line
    errs = np.zeros((n_iter, 3)) #One column per error type
    
    theta = y.copy()
    
    times = [0]*n_iter
    t = 0.0 #times
    theta_out = theta.copy()
    for i in range(n_iter):
        
        #samples
        times[i] = t
        samples_theta[i] = theta_out

        #errors
        err_quad = 1. / 2. * np.sum((y - theta)**2)
        err_reg = lmbda  * np.sum(np.abs(np.dot(grad,theta)))

        errs[i] = err_quad + err_reg, err_quad, err_reg
        
        #algo
        std = np.sqrt(2 * gamma) * float(gaussian_noise)
        top = time.time()
        
        theta = proxtv((theta + gamma*temp*y)/(1.0 + gamma*temp), grad, lmbda*gamma*temp / (1.0 + gamma*temp), itprox, pasprox) 
        theta_out = theta.copy()
        theta += std * np.random.randn(N)  
        
        tip = time.time() - top
        t += tip
        
    return samples_theta, errs, times


def TV(x,G):
    r = 0
    for e in G.Edges():
        r += np.abs(x[e.GetSrcNId()] - x[e.GetDstNId()])
    return r


def passty_langevin_sto(Y, n_it, gamma, lmbda, G, b, L, temp, par, gaussian_noise=True):
    """ Stochastic_Passty_Algorithm
    Inputs:
        temp: temperature
        n_iter: number of iterations
        Y: Matrix of signals on the original graph
        gamma: constant step size
        lmbda: weight of regularization
        b : minibatch size for the likelihood
        L : minibatch size for the prior
        grad : graph gradient matrix
        l : TV estimation
        gaussian_noise: if False passty algorithm is applied, else Passty-Langevin
     Output:
         samples
    """
    V, E = G.GetMxNId(), G.GetEdges() #number of edges and vertices
    D = Y.shape[0] #number of data

    samples_theta = np.zeros((n_it, V)) #iterates stacked by line
    errs = np.zeros((n_it, 3)) #One column per error type
    
    theta = np.mean(Y,axis = 0) #Start with the mean of observations
    t = 0.0 #time
    std = np.sqrt(2 * gamma) * float(gaussian_noise)
    times = [0]*n_it
    theta_out = theta.copy()
    for i in range(n_it):
        if i % par == 0: 
            print i 
        times[i] = t
        samples_theta[i] = theta_out
        #Algo
        
        top = time.time() 
        for j in range(L):
            #print j
            e = G.GetRndEId()
            e = G.GetEI(e)
            theta = prox_edge(theta, temp * lmbda * gamma * E/float(L), e.GetSrcNId(), e.GetDstNId())
        theta_out = theta.copy()
        theta -= temp * gamma * np.mean([theta - Y[np.random.randint(D),:] for j in range(b)])
        theta += std * np.random.randn(V)
        tip = time.time() - top
        t += tip
    return samples_theta, times, errs

def subgrad_langevin_sto(Y, n_it, gamma, lmbda, G, b, L, temp, par, gaussian_noise=True):
    """ Stochastic_Passty_Algorithm
    Inputs:
        n_iter: number of iterations
        Y: Matrix of signals on the original graph
        gamma: constant step size
        lmbda: weight of regularization
        b : minibatch size for the likelihood
        L : minibatch size for the prior
        grad : graph gradient matrix
        l : TV estimation
        gaussian_noise: if False passty algorithm is applied, else Passty-Langevin
     Output:
         samples
    """
    V, E = G.GetMxNId(), G.GetEdges() #number of edges and vertices
    D = Y.shape[0] #number of data

    samples_theta = np.zeros((n_it, V)) #iterates stacked by line
    errs = np.zeros((n_it, 3)) #One column per error type
    
    theta = np.mean(Y, axis = 0) #Start with the mean of observations
    t = 0.0 #time
    std = np.sqrt(2 * gamma / E) * float(gaussian_noise)
    times = [0]*n_it
    for i in range(n_it):
        if i % par == 0: 
            print i
        times[i] = t
        samples_theta[i] = theta
        
        top = time.time() 
        for j in range(L):
            e = G.GetRndEId()
            e = G.GetEI(e)
            theta[e.GetSrcNId()] -= temp * lmbda * gamma * E/float(L) * np.sign(theta[e.GetSrcNId()] - theta[e.GetDstNId()])
            theta[e.GetDstNId()] -= temp * lmbda * gamma * E/float(L) * np.sign(theta[e.GetDstNId()] - theta[e.GetSrcNId()])
        theta -= temp * gamma * np.mean([theta - Y[np.random.randint(D),:] for j in range(b)])
        theta += std * np.random.randn(V)
        tip = time.time() - top
        t += tip
    return samples_theta, times, errs




# Parameters

In [ ]:
#General parameters

D = 1
b = 1


In [ ]:
#Figure 1, FB gaussian


G = snap.LoadEdgeList(snap.PNEANet, "facebook_combined.txt", 0, 1)
N = G.GetNodes()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight
Y = np.random.randn(D,N)
Y.shape

alpha = 50 * float(M) 
gamma = .1 * float(N) / alpha 
temp = alpha/float(M) 
L = 400 
grad = gradient(G)
par_spla = 100
par_ssla = par_spla * L / 80


n_it = 1500
nb_samples = 5
n_it_sub = n_it * L / 80

gamma_prox = .5 / temp
n_iter_pla = 14

In [ ]:
#Figure 2, FB inpainting


G = snap.LoadEdgeList(snap.PNEANet, "facebook_combined.txt", 0, 1)
N = G.GetNodes()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight

Y = np.random.randn(D,N)
Y.shape

for i in range(N/2):
    Y[0,i] = 0.0

alpha = 50 * float(M) 
gamma = .1 * float(N) / alpha 
temp = alpha/float(M) 
L = 400 

grad = gradient(G)

par_spla = 100

par_ssla = par_spla * L / 80


n_it = 1500
nb_samples = 5

n_it_sub = n_it * L / 80


gamma_prox = .5 / temp
n_iter_pla = 14


In [ ]:
#Figure 3, Youtube gaussian


G = snap.LoadEdgeList(snap.PNEANet, "com-youtube.ungraph.txt", 0, 1)
N = G.GetMxNId()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight

Y = np.random.randn(D,N)
Y.shape



alpha = 50 * float(M) 
gamma = .001 * float(N) / alpha 
temp = alpha/float(M) 
L = 400 

par_spla = 1


par_ssla = par_spla * L / 20


n_it = 15 
nb_samples = 5

n_it_sub = n_it * L / 20

In [ ]:
#Figure 4, Youtube inpainting


G = snap.LoadEdgeList(snap.PNEANet, "com-youtube.ungraph.txt", 0, 1)
N = G.GetMxNId()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight

Y = np.random.randn(D,N)
Y.shape


for i in range(N/2):
    Y[0,i] = 0.0

alpha = 50 * float(M) 
gamma = .001 * float(N) / alpha 
temp = alpha/float(M) 
L = 4000 

par_spla = 1


par_ssla = par_spla * L / 80


n_it = 15
nb_samples = 5

n_it_sub = n_it * 15


In [ ]:
#Figure 5-6, DBLP


G = snap.LoadEdgeList(snap.PNEANet, "com-dblp.ungraph.txt", 0, 1)
N = G.GetMxNId()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight

Y = np.random.randn(D,N)
Y.shape

alpha = 50 * float(M) 
gamma = .01 * float(N) / alpha 
temp = alpha/float(M) 
L = 400

par_spla = 1


par_ssla = par_spla * L / 80


n_it = 15
nb_samples = 5

n_it_sub = n_it * L / 80


In [ ]:
#Figure 7-8, Amazon


G = snap.LoadEdgeList(snap.PNEANet, "com-amazon.ungraph.txt", 0, 1)
N = G.GetMxNId()
M = G.GetEdges()
print N,M

lmbda = N * np.sqrt(np.pi) / (2*M)
print lmbda * TV(np.random.randn(N),G)/(0.5 * np.linalg.norm(np.random.randn(N) - np.random.randn(N))**2) #Same weight

Y = np.random.randn(D,N)
Y.shape

alpha = 50 * float(M) 
gamma = .01 * float(N) / alpha 
temp = alpha/float(M) 
L = 400

par_spla = 1


par_ssla = par_spla * L / 80


n_it = 15
nb_samples = 5

n_it_sub = n_it * L / 80


# Simulations

# SPLA

In [ ]:
t0 = time.time()
samples = []
times_mean = []


for i in range(nb_samples):
    thetas, times, errs = passty_langevin_sto(Y, n_it, gamma, lmbda, G, b, L, temp, par_spla, gaussian_noise=True)
    samples += [thetas]
    times_mean += [times]
    
print time.time() - t0
print times[-1]

y = Y[0,:]

#Minimization KL
samples_mix = []
for i in range(nb_samples):
    thetas_mix = []
    for j in range(n_it):
        k = np.random.randint(j+1)
        thetas_mix += [samples[i][k,:]] #mixing
    samples_mix += [thetas_mix]

    
A = np.asarray(times_mean)
t_pass = np.mean(A,axis=0)
t_pass = list(t_pass)    

#KL estimation


KL = []
time_passty = []
for k in range(n_it):
    if k % par_spla == 0: 
        print k
        time_passty += [t_pass[k]]
        Theta_k = np.asarray([samples_mix[i][k] for i in range(nb_samples)]) #each line is a realization of SPLA after mixing
        H_k = Entropy(Theta_k) 
        E_k = np.mean([U(Theta_k[i,:],G,temp,lmbda) for i in range(nb_samples)])
        KL += [H_k + E_k]

# Stochastic Subgradient

In [ ]:
#We need a small step size for this method to work
t0 = time.time()
samples_sub = []
times_mean_sub = []

for i in range(nb_samples):
    thetas, times, errs = subgrad_langevin_sto(Y, n_it_sub, gamma, lmbda, G, b, L, temp, par_ssla, gaussian_noise=True)
    samples_sub += [thetas]
    times_mean_sub += [times]
    
print time.time() - t0
print times[-1]

y = Y[0,:]

samples_mix_sub = []
for i in range(nb_samples):
    thetas_mix_sub = []
    for j in range(n_it_sub):
        k = np.random.randint(j+1)
        thetas_mix_sub += [samples_sub[i][k,:]] 
    samples_mix_sub += [thetas_mix_sub]


B = np.asarray(times_mean_sub)
t_sub = np.mean(B,axis=0)
t_sub = list(t_sub)

time_sub = []
KL_sub = []
for k in range(n_it_sub):
    if k % par_ssla == 0:
        print k
        time_sub += [t_sub[k]]
        Theta_k = np.asarray([samples_mix_sub[i][k] for i in range(nb_samples)]) 
        H_k = Entropy(Theta_k) 
        E_k = np.mean([U(Theta_k[i,:],G,temp,lmbda) for i in range(nb_samples)])
        KL_sub += [H_k + E_k]
        

# ProxLA

In [ ]:
y = Y[0,:]
t0 = time.time()
samples_pla = []
times_mean_pla = []

for i in range(nb_samples):
    thetas_pla, errs_pla, times_pla = PLA(y, n_iter_pla, gamma_prox, lmbda, temp, grad, itprox = 10, pasprox = 0.001, gaussian_noise=True)
    samples_pla += [thetas_pla]
    times_mean_pla += [times_pla]
    
print time.time() - t0

samples_mix_pla = []
for i in range(nb_samples):
    thetas_mix_pla = []
    for j in range(n_iter_pla):
        k = np.random.randint(j+1)
        thetas_mix_pla += [samples_pla[i][k,:]] 
    samples_mix_pla += [thetas_mix_pla]


C = np.asarray(times_mean_pla)
t_pla = np.mean(C,axis=0)
t_pla = list(t_pla)


KL_pla = []

for k in range(n_iter_pla):
    Theta_k = np.asarray([samples_mix_pla[i][k] for i in range(nb_samples)]) 
    H_k = Entropy(Theta_k)
    E_k = np.mean([U(Theta_k[i,:],G,temp,lmbda) for i in range(nb_samples)])
    KL_pla += [H_k + E_k]
    

# Resultats

In [ ]:
%matplotlib inline

p1 = round(1/temp,2)
p2 = round(temp * lmbda,1)
p3 = round(gamma,6)
#p4 = round(gamma_prox,2)

passty_time = plt.plot(time_passty, KL, drawstyle='Default',marker = 'None', label = 'SPLA, $\gamma$ = '+ str(p3))
#pla_time = plt.plot(t_pla, KL_pla, drawstyle='steps-post',marker = 's', label = 'ProxLA, $\gamma$ = '+ str(p4))
sub_time = plt.plot(time_sub, KL_sub, drawstyle='default',marker = 'None', label = 'SSLA, $\gamma$ = '+ str(p3), linestyle = '--')



plt.xlabel('CPU time (in seconds)')
plt.ylabel('$E+H$')
plt.title('$E+H$ as a function of time, $\sigma^2$ = '+str(p1)+' $, \lambda$ = '+str(p2))
fontP = FontProperties()
legend = plt.legend(loc = 0, shadow = True, fontsize = 'x-large')
legend.get_frame().set_facecolor('#efe2c1')
plt.axis('tight')
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.grid()
#plt.savefig('Langevin-Fig3-v2.png')
plt.show()
